In [1]:
import pandas as pd
from sqlanalyzer import column_parser
import re

In [2]:
query = """"SELECT sfdc_accounts.platform,           sfdc_accounts.mobile_os,           sfdc_accounts.service_metadata,           sfdc_cases.account,           sfdc_cases.num_requests,           sfdc_cases.owner,           sfdc_accounts.user_id    FROM sfdc.accounts sfdc_accounts    LEFT JOIN sfdc.cases sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id    WHERE sfdc_cases_oppty.dt > '2020-04-03'      AND sfdc_cases_oppty.dt < '2020-05-04'    ORDER BY 1    GROUP BY 3 LIMIT 20"
"""

In [3]:
db_fields_1 = pd.DataFrame({'db_table': 'sfdc.cases', 
            'all_columns': ['account', 'num_requests', 'owner', 'anonymous_id', 'id', 'original_timestamp', 'event', 'context_traits_email', 'service_metadata_version', 'dt']})
db_fields_2 = pd.DataFrame({'db_table': 'sfdc.accounts',
                           'all_columns': ['platform', 'case_id', 'mobile_os', 'service_metadata', 'user_id', 'first_name', 'last_name']})
df = db_fields_1.append(db_fields_2, ignore_index=True)

In [4]:
df

,db_table,all_columns
0,sfdc.cases,account
1,sfdc.cases,num_requests
2,sfdc.cases,owner
3,sfdc.cases,anonymous_id
4,sfdc.cases,id
5,sfdc.cases,original_timestamp
6,sfdc.cases,event
7,sfdc.cases,context_traits_email
8,sfdc.cases,service_metadata_version
9,sfdc.cases,dt


In [5]:
formatter = column_parser.Parser(query)
formatted = formatter.format_query(query)
formatted

"SELECT sfdc_accounts.platform,\n       sfdc_accounts.mobile_os,\n       sfdc_accounts.service_metadata,\n       sfdc_cases.account,\n       sfdc_cases.num_requests,\n       sfdc_cases.owner,\n       sfdc_accounts.user_id\nFROM sfdc.accounts sfdc_accounts\nLEFT JOIN sfdc.cases sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id\nWHERE sfdc_cases_oppty.dt > '2020-04-03'\n  AND sfdc_cases_oppty.dt < '2020-05-04'\nORDER BY 1\nGROUP BY 3\nLIMIT 20"

In [6]:
fields = formatter.match_queried_fields(formatted, df)
fields

[{'database_name': 'sfdc', 'table_name': 'cases', 'column_name': 'id'},
 {'database_name': 'sfdc',
  'table_name': 'cases',
  'column_name': 'num_requests'},
 {'database_name': 'sfdc', 'table_name': 'cases', 'column_name': 'account'},
 {'database_name': 'sfdc', 'table_name': 'cases', 'column_name': 'owner'}]

In [33]:
all_columns_scanned = []

for _,cte_query in cte_queries.items():

    table_alias_mapping = formatter.get_table_names(cte_query.split('\n'))
    variables = formatter._get_all_variables(cte_query)
    queried_columns = formatter._get_queried_columns(table_alias_mapping, df)
    print(queried_columns)
    if variables == []:
        original_columns_list = []
        for table in queried_columns:
            for k,v in table.items():
                for t in v:
                    original_columns_list.append("{}.{}".format(k,t))
    else:
        print(formatter._map_db_columns(variables, queried_columns, table_alias_mapping))
#         all_columns_scanned.extend(formatter._map_db_columns(variables, queried_columns, table_alias_mapping))
        
#     all_columns_scanned.extend(list(set(original_columns_list)))

[{'sfdc.accounts': {'user_id', 'last_name', 'service_metadata', 'mobile_os', 'first_name', 'case_id', 'platform'}}, {'sfdc.cases': {'anonymous_id', 'service_metadata_version', 'id', 'num_requests', 'original_timestamp', 'event', 'context_traits_email', 'dt', 'account', 'owner'}}]
['sfdc.cases.account', 'sfdc.cases.num_requests', 'sfdc.cases.id', 'sfdc.cases.owner']


In [36]:
cte_query = cte_queries['main']
cte_query

"SELECT sfdc_accounts.platform,\n       sfdc_accounts.mobile_os,\n       sfdc_accounts.service_metadata,\n       sfdc_cases.account,\n       sfdc_cases.num_requests,\n       sfdc_cases.owner,\n       sfdc_accounts.user_id\nFROM sfdc.accounts sfdc_accounts\nLEFT JOIN sfdc.cases sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id\nWHERE sfdc_cases_oppty.dt > '2020-04-03'\n  AND sfdc_cases_oppty.dt < '2020-05-04'\nORDER BY 1\nGROUP BY 3\nLIMIT 20"

In [40]:
table_alias_mapping = formatter.get_table_names(cte_query.split('\n'))
variables = formatter._get_all_variables(cte_query)
queried_columns = formatter._get_queried_columns(table_alias_mapping, df)
variables

['sfdc_accounts.platform',
 'sfdc_accounts.mobile_os',
 'sfdc_accounts.service_metadata',
 'sfdc_cases.account',
 'sfdc_cases.num_requests',
 'sfdc_cases.owner',
 'sfdc_accounts.user_id',
 'sfdc.accounts sfdc_accounts',
 '',
 'sfdc.cases sfdc_cases',
 'sfdc_cases.id',
 'sfdc_accounts.case_id',
 'sfdc_cases_oppty.dt',
 '',
 '',
 'sfdc_cases_oppty.dt',
 '',
 '',
 '',
 '',
 '',
 '']

In [43]:
# formatter._map_db_columns(variables, queried_columns, table_alias_mapping)
var_list, queried_cols, table_alias_mapping = variables, queried_columns, table_alias_mapping
original_columns_list = []

for var in set(var_list):

    var = var.strip(' ')

    if var in table_alias_mapping.keys():
        pass

    else:
        var_split = var.split('.')
        print(var_split)
        if len(var_split) == 1:
            for db_table in queried_cols:
                for k,v in db_table.items():
                    if var in v:
                        original_columns_list.append("{}.{}".format(k, var))

        elif len(var_split) == 2:

            if var_split[0] in table_alias_mapping.keys():
                db_table = table_alias_mapping[var_split[0]]

                for db_table_col in queried_cols:
                    for k,v in db_table_col.items():
                        if k == db_table and var_split[1] in v:
                            original_columns_list.append("{}.{}".format(k, var_split[1]))

['']
['sfdc_accounts', 'service_metadata']
['sfdc_cases', 'num_requests']
['sfdc_cases', 'owner']
['sfdc_accounts', 'case_id']
['sfdc_accounts', 'user_id']
['sfdc_cases', 'id']
['sfdc_cases_oppty', 'dt']
['sfdc', 'cases sfdc_cases']
['sfdc_accounts', 'platform']
['sfdc_cases', 'account']
['sfdc_accounts', 'mobile_os']
['sfdc', 'accounts sfdc_accounts']


In [28]:
all_columns_scanned

['sfdc.cases.account',
 'sfdc.cases.num_requests',
 'sfdc.cases.id',
 'sfdc.cases.owner']

In [11]:
line_query = formatted.split('\n')
line_query

['SELECT sfdc_accounts.platform,',
 '       sfdc_accounts.mobile_os,',
 '       sfdc_accounts.service_metadata,',
 '       sfdc_cases.account,',
 '       sfdc_cases.num_requests,',
 '       sfdc_cases.owner,',
 '       sfdc_accounts.user_id',
 'FROM sfdc.accounts sfdc_accounts',
 'LEFT JOIN sfdc.cases sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id',
 "WHERE sfdc_cases_oppty.dt > '2020-04-03'",
 "  AND sfdc_cases_oppty.dt < '2020-05-04'",
 'ORDER BY 1',
 'GROUP BY 3',
 'LIMIT 20']

In [12]:
# get_table_names
table_name_mapping = dict()

for line in line_query:

    table_line = re.findall(r"(FROM|JOIN).(\w+.*)", line)

    if table_line != []:
        table_name_line = table_line[0][1].split(' ')
        print(table_name_line)
        if len(table_name_line) <= 2:
            table_name_mapping[table_name_line[0].rstrip(')|,')] = table_name_line[0].rstrip(')|,')

        elif len(table_name_line) > 2 and table_name_line[1] == 'AS':
            table_name_mapping[table_name_line[2].rstrip(')|,')] = table_name_line[0].rstrip(')|,')
        
        elif len(table_name_line) > 2 and table_name_line[1] != 'AS':
            table_name_mapping[table_name_line[1].rstrip(')|,')] = table_name_line[0].rstrip(')|,')
            

['sfdc.accounts', 'sfdc_accounts']
['sfdc.cases', 'sfdc_cases', 'ON', 'sfdc_cases.id', '=', 'sfdc_accounts.case_id']


In [13]:
table_name_mapping

{'sfdc.accounts': 'sfdc.accounts', 'sfdc_cases': 'sfdc.cases'}

In [14]:
formatted_query = formatter.format_query(query)
cte_queries = formatter.parse_cte(formatted_query)
all_columns_scanned = formatter._get_all_scanned_cols(cte_queries, df)
all_columns_scanned


['sfdc.cases.id',
 'sfdc.cases.num_requests',
 'sfdc.cases.account',
 'sfdc.cases.owner']

In [ ]:
column_payload = []
for column in all_columns_scanned:
    try:
        col_split = column.split('.')
        db, table, col = col_split[0], col_split[1], col_split[2]
        row_payload = dict()
        row_payload["database_name"] = db
        row_payload["table_name"] = table 
        row_payload["column_name"] = col
        for arg, value in kwargs.items():
            row_payload[arg] = value    

        column_payload.append(row_payload)  
    except:
        pass